Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.6653345369377348e-16 - 0.26835469053190525im
-8.630249292984615e-17 + 0.2683546905319051im
0.03747531973228935 + 0.9999999999999997im
-0.037475319732288806 + 1.0im

Floquet Multipliers = 

0.6651802570066604 - 0.7466828146465888im
0.665180257006661 + 0.7466828146465889im
-1.1249427991479373 + 1.1369156774202889e-15im
-0.8889340869219566 + 1.0886302842019492e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,109.226 596.058,109.204 596.049,109.161 596.035,109.095 596.017,109.007 595.994,108.898 595.966,108.768 595.933,108.616 595.896,108.443 595.855,108.249 
 595.808,108.034 595.757,107.8 595.702,107.546 595.642,107.272 595.577,106.979 595.507,106.667 595.433,106.338 595.355,105.991 595.271,105.627 595.183,105.247 
 595.091,104.851 594.994,104.441 594.892,104.016 594.785,103.578 594.674,103.128 594.559,102.665 594.439,102.193 594.314,101.71 594.184,101.219 594.05,100.721 
 593.912,100.216 593.768,99.7066 593.621,99.193 593.468,98.6769 593.311,98.1596 593.15,97.6427 592.983,97.1276 592.813,96.6158 592.637,96.1091 592.458,95.609 
 592.273,95.1174 592.084,94.6359 591.89,94.1665 591.692,93.7112 591.489,93.2717 591.282,92.8503 591.07,92.449 590.854,92.07 590.633,91.7153 590.408,91.3874 
 590.178,91.0886 589.943,90.821 589.704,90.5873 589.46,90.3898 589.212,90.231 588.959,90.1134 588.702,90.0396 588.441,90.0121 588.174,90.0337 587.904,90.1068 
 587.629,90.2341 587.349,90.4183 587.065,90.662 586.776,90.9678 586.483,91.3384 586.185,91.7764 585.883,92.2843 585.577,92.8647 585.266,93.5202 584.95,94.253 
 584.631,95.0658 584.306,95.9606 583.977,96.9399 583.644,98.0057 583.307,99.1601 582.965,100.405 582.618,101.742 582.267,103.174 581.912,104.701 581.552,106.325 
 581.188,108.047 580.82,109.869 580.447,111.79 580.07,113.813 579.688,115.937 579.302,118.163 578.912,120.49 578.518,122.919 578.119,125.449 577.715,128.08 
 577.308,130.809 576.896,133.637 576.48,136.562 576.059,139.581 575.634,142.694 575.205,145.896 574.772,149.186 574.334,152.561 573.892,156.016 573.446,159.55 
 572.995,163.156 572.54,166.832 572.081,170.572 571.618,174.37 571.151,178.223 570.679,182.124 570.203,186.066 569.723,190.044 569.239,194.051 568.75,198.078 
 568.258,202.12 567.761,206.168 567.26,210.213 566.755,214.248 566.246,218.264 565.732,222.251 565.215,226.2 564.693,230.102 564.167,233.946 563.637,237.724 
 563.104,241.423 562.565,245.034 562.023,248.547 561.477,251.95 560.927,255.232 560.373,258.383 559.814,261.392 559.252,264.247 558.686,266.939 558.115,269.455 
 557.541,271.786 556.963,273.921 556.38,275.85 555.794,277.563 555.204,279.051 554.609,280.305 554.011,281.317 553.409,282.078 552.803,282.581 552.193,282.82 
 551.579,282.79 550.962,282.485 550.34,281.902 549.714,281.037 549.085,279.89 548.452,278.458 547.815,276.743 547.174,274.747 546.529,272.472 545.881,269.923 
 545.228,267.106 544.572,264.027 543.913,260.695 543.249,257.12 542.582,253.314 541.91,249.288 541.236,245.057 540.557,240.637 539.875,236.043 539.189,231.293 
 538.499,226.407 537.806,221.405 537.109,216.307 536.408,211.134 535.704,205.91 534.996,200.658 534.284,195.4 533.569,190.16 532.851,184.963 532.128,179.832 
 531.402,174.792 530.673,169.865 529.94,165.074 529.203,160.442 528.463,155.991 527.72,151.74 526.973,147.711 526.222,143.921 525.468,140.388 524.711,137.127 
 523.95,134.153 523.186,131.478 522.418,129.113 521.647,127.068 520.872,125.35 520.094,123.965 519.313,122.916 518.529,122.204 517.741,121.83 516.949,121.79 
 516.155,122.082 515.357,122.698 514.556,123.63 513.751,124.869 512.943,126.402 512.132,128.214 511.318,130.291 510.501,132.616 509.68,135.167 508.856,137.926 
 508.029,140.869 507.199,143.974 506.366,147.214 505.529,150.563 504.689,153.996 503.847,157.482 503.001,160.993 502.152,164.5 501.3,167.973 500.445,171.382 
 499.587,174.697 498.726,177.888 497.862,180.927 496.994,183.785 496.124,186.435 495.251,188.852 494.375,191.011 493.496,192.892 492.614,194.474 491.73,195.74 
 490.842,196.675 489.951,197.27 489.058,197.514 488.162,197.405 487.262,196.941 486.361,196.124 485.456,194.961 484.548,193.462 483.638,191.641 482.725,189.516 
 481.809,187.108 480.89,184.442 479.969,181.5